In [2]:
from dotenv import load_dotenv
from utils.agent_visualizer import print_activity, visualize_conversation

from claude_agent_sdk import ClaudeAgentOptions, ClaudeSDKClient

load_dotenv()

False

# 01 - 首席助理智能体

#### 介绍

在笔记本00中，我们构建了一个简单的研究智能体。在本笔记本中，我们将逐步介绍构建全面智能体的关键Claude Code SDK功能。对于每个介绍的功能，我们将解释：
- **什么**：功能是什么
- **为什么**：该功能可以做什么以及为什么您想使用它
- **如何**：展示如何使用它的最小实现

如果您熟悉Claude Code，您会注意到SDK如何带来功能对等性，并使您能够以编程方式无头地利用Claude Code的所有功能。

#### 场景

在本笔记本中，我们将为一个刚刚获得1000万美元A轮融资的50人初创公司构建一个**AI首席助理**。CEO需要数据驱动的洞察来平衡激进增长与财务可持续性。

我们的最终首席助理智能体将：
- **协调专门子智能体**处理不同领域
- **汇总**来自多个来源的洞察
- **提供**具有可操作建议的执行摘要

## 基础功能

### 功能0：使用[CLAUDE.md](https://www.anthropic.com/engineering/claude-code-best-practices)的记忆

**什么**：`CLAUDE.md`文件用作智能体的持久性记忆和指令。当存在于项目目录中时，Claude Code在初始化您的智能体时会自动读取并合并此上下文。

**为什么**：无需在每次交互中重复提供项目上下文、团队偏好或标准，您可以在`CLAUDE.md`中一次性定义它们。这确保了一致性行为，并通过避免冗余解释来减少令牌使用量。

**如何**：
- 在工作目录中有一个`CLAUDE.md`文件 - 在我们的示例中：`chief_of_staff_agent/CLAUDE.md`
- 将您的ClaudeSDKClient的`cwd`参数设置为指向您的CLAUDE.md文件的目录

In [ ]:
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="chief_of_staff_agent",  # Points to subdirectory with our CLAUDE.md
    )
) as agent:
    await agent.query("What's our current runway?")
    async for msg in agent.receive_response():
        if hasattr(msg, "result"):
            print(msg.result)
# The agent should know from the CLAUDE.md file: $500K burn, 20 months runway

### 功能1：Bash工具用于Python脚本执行

**什么**：Bash工具允许您的智能体（除其他功能外）直接运行Python脚本，从而能够访问程序化知识、复杂计算、数据分析以及其他超越智能体原生能力的集成。

**为什么**：我们的首席助理可能需要处理数据文件、运行基于此数据的财务模型或生成可视化。这些都是使用Bash工具的好场景。

**如何**：将您的Python脚本设置在智能体可以访问的地方，并添加关于它们是什么以及如何调用它们的上下文。如果脚本是为您的主助理智能体准备的，将此上下文添加到其CLAUDE.md文件中，如果它们是为您的一个子智能体准备的，将所述上下文添加到它们的MD文件中（稍后会更详细地介绍）。对于本教程，我们在`chief_of_staff_agent/scripts`中添加了五个玩具示例：
1. `hiring_impact.py`：计算新工程招聘如何影响燃烧率、跑道和现金状况。对`financial-analyst`子智能体根据50万美元月燃烧率和20个月跑道建模招聘场景至关重要。
2. `talent_scorer.py`：使用加权标准对候选人的技术技能、经验、文化契合度和薪资期望进行评分。`recruiter`子智能体根据TechStart的18-22万美元高级工程师基准对工程候选人进行排名的核心工具。
3. `simple_calculation.py`：对跑道、燃烧率和季度指标执行快速财务计算。首席助理获取即时指标而无需复杂建模的实用脚本。
4. `financial_forecast.py`：对当前240万美元ARR以15%月环比增长建模ARR增长场景（基础/乐观/悲观）。对`financial-analyst`预测B轮准备状态并验证3000万美元融资目标至关重要。
5. `decision_matrix.py`：为诸如SmartDev收购或办公室扩张等战略选择创建加权决策矩阵。帮助首席助理系统地评估具有多个利益相关者和标准的复杂决策。

In [ ]:
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        allowed_tools=["Bash", "Read"],
        cwd="chief_of_staff_agent",  # Points to subdirectory where our agent is defined
    )
) as agent:
    await agent.query(
        "Use your simple calculation script with a total runway of 2904829 and a monthly burn of 121938."
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        if hasattr(msg, "result"):
            print("\n")
            print(msg.result)

### 功能2：输出样式

**什么**：输出样式允许您为不同受众使用不同的输出样式。每种样式都在一个markdown文件中定义。

**为什么**：您的智能体可能被具有不同专业水平的人使用，或者他们可能有不同的优先级。您的输出样式可以帮助在这些细分之间进行区分，而无需创建单独的智能体。

**如何**：
- 在`chief_of_staff_agent/.claude/output-styles/`中为每种样式配置一个markdown文件。例如，查看`.claude/output-styles/executive.md`中的执行输出样式。输出样式使用简单的前言定义，包括两个字段：名称和描述。注意：确保前言中的名称与文件名完全匹配（区分大小写）

> **重要**：输出样式修改Claude Code底层的系统提示，去除专注于软件工程的部分，让您对软件工程工作之外的特定用例有更多控制。

In [3]:
messages_executive = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="chief_of_staff_agent",
        settings='{"outputStyle": "executive"}',
    )
) as agent:
    await agent.query("Tell me in two sentences about your writing output style.")
    async for msg in agent.receive_response():
        print_activity(msg)
        messages_executive.append(msg)

messages_technical = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="chief_of_staff_agent",
        settings='{"outputStyle": "technical"}',
    )
) as agent:
    await agent.query("Tell me in two sentences about your writing output style.")
    async for msg in agent.receive_response():
        print_activity(msg)
        messages_technical.append(msg)

🤖 Thinking...
🤖 Thinking...


In [ ]:
print(messages_executive[-1].result)

In [ ]:
print(messages_technical[-1].result)

### 功能3：计划模式 - 无执行的战略规划

**什么**：计划模式指示智能体创建详细的执行计划而不执行任何操作。智能体分析需求、提出解决方案并概述步骤，但不修改文件、执行命令或进行更改。

**为什么**：复杂任务受益于预先规划，以减少错误、实现审查并改善协调。在规划阶段之后，智能体将在整个执行过程中有一条红线遵循。

**如何**：只需设置`permission_mode="plan"`

> 注意：此功能在Claude Code中闪闪发光，但仍然需要针对带SDK的无头应用程序进行充分适应。也就是说，智能体将尝试调用其`ExitPlanMode()`工具，该工具仅在交互模式下相关。在这种情况下，您可以发送带有`continue_conversation=True`的后续查询，以便智能体在上下文中执行其计划。

In [9]:
messages = []
async with (
    ClaudeSDKClient(
        options=ClaudeAgentOptions(
            model="claude-opus-4-1",  # We're using Opus for this as Opus truly shines when it comes to planning!
            permission_mode="plan",
        )
    ) as agent
):
    await agent.query("Restructure our engineering team for AI focus.")
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Using: Glob()
✓ Tool completed
🤖 Using: Read()
✓ Tool completed
🤖 Using: Glob()
✓ Tool completed
🤖 Using: Read()
✓ Tool completed
🤖 Using: Read()
✓ Tool completed
🤖 Using: Glob()
✓ Tool completed
🤖 Thinking...
🤖 Using: ExitPlanMode()
✓ Tool completed


In [10]:
print(messages[-1].result)

如上所述，智能体将在创建其计划后停止，如果希望它执行其计划，您需要发送带有`continue_conversation=True`的新查询并删除`permission_mode="plan"`

## 高级功能

### 功能4：自定义斜杠命令

> 注意：斜杠命令是用户的语法糖，不是新的智能体功能

**什么**：自定义斜杠命令是用户可以使用简写语法（例如`/budget-impact`）触发的预定义提示模板。这些是**面向用户的快捷方式**，不是智能体功能。将它们视为扩展为完整的、精心制作的提示的键盘快捷键。

**为什么**：您的主助理将处理重复出现的执行问题。用户无需重复输入复杂提示，可以使用已经审查的提示。这改善了一致性和标准化。

**如何**：
- 在`.claude/commands/`中定义一个markdown文件。例如，我们在`.claude/commands/slash-command-test.md`中定义了一个。注意命令是如何定义的：具有两个字段（名称、描述）的前言和带有在查询中传递的参数选项的扩展提示。
- 您可以使用`{{args}}`向提示添加参数
- 用户在提示中使用斜杠命令

In [11]:
# User types: "/slash-command-test this is a test"
# -> behind the scenes EXPANDS to the prompt in .claude/commands/slash-command-test.md
# In this case the expanded prompt says to simply reverse the sentence word wise

messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(model="claude-sonnet-4-5", cwd="chief_of_staff_agent")
) as agent:
    await agent.query("/slash-command-test this is a test")
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...


In [12]:
print(messages[-1].result)

test a is this


### 功能5：钩子 - 自动化确定性操作

**什么**：钩子是您可以设置为在特定工具调用之前（前）或之后（后）自动执行的Python脚本以及其他事件。钩子**确定性运行**，使它们非常适合验证和审计跟踪。

**为什么**：想象一下您想要确保您的智能体有一些护栏（例如，防止危险操作）或当您想要有审计跟踪的场景。钩子与智能体结合是理想的，允许它们有足够的自由来实现其任务，同时仍确保智能体以安全的方式行为。

**如何**：
- 在`.claude/hooks/`中定义钩子脚本 -> 钩子触发时应执行的内容是什么
- 在`.claude/settings.local.json`中定义钩子配置 -> 何时应触发钩子
- 在这种情况下，我们的钩子被配置为监视特定工具调用（WebSearch、Write、Edit等）
- 当调用这些工具时，钩子脚本要么首先运行（前工具使用钩子）要么之后运行（后工具使用钩子）

**示例：合规性报告跟踪**

用于记录财务报告上的Write/Edit操作以进行审计和合规目的的钩子。
钩子定义在`chief_of_staff_agent/.claude/hooks/report-tracker.py`中，强制执行的逻辑在`chief_of_staff/.claude/settings.local.json`中：

In [13]:
messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="chief_of_staff_agent",
        allowed_tools=["Bash", "Write", "Edit", "MultiEdit"],
    )
) as agent:
    await agent.query(
        "Create a quick Q2 financial forecast report with our current burn rate and runway projections. Save it to our /output_reports folder."
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

# The hook will track this in audit/report_history.json

🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: Bash()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: Bash()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: Write()
✓ Tool completed
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...


如果您现在导航到`./chief_of_staff_agent/audit/report_history.json`，您会发现它记录了智能体已创建和/或对您的报告进行更改。您可以在`./chief_of_staff_agent/output_reports/`中找到生成的报告本身。

### 功能6：通过任务工具的子智能体

**什么**：任务工具使您的智能体能够将专门的工作委托给其他子智能体。这些子智能体每个都有自己的指令、工具和专业知识。

**为什么**：添加子智能体开辟了很多可能性：
1. 专业化：每个子智能体都是其领域的专家
2. 独立上下文：子智能体有自己的对话历史和工具
3. 并行化：多个子智能体可以同时处理不同方面。

**如何**：
- 将"Task"添加到allowed_tools
- 使用系统提示指示您的智能体如何委派任务（您也可以在CLAUDE.md中更一般地定义这一点）
- 在`.claude/agents/`中为每个智能体创建一个markdown文件。例如，查看`.claude/agents/financial-analyst.md`并注意如何用如此简单和直观的markdown文件定义一个（子）智能体：具有三个字段（名称、描述和工具）的前言及其系统提示。描述对主首席助理智能体知道何时调用每个子智能体很有用。

In [14]:
messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        allowed_tools=["Task"],  # this enables our Chief agent to invoke subagents
        system_prompt="Delegate financial questions to the financial-analyst subagent. Do not try to answer these questions yourself.",
        cwd="chief_of_staff_agent",
    )
) as agent:
    await agent.query("Should we hire 5 engineers? Analyze the financial impact.")
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: Task()
🤖 Using: Bash()
🤖 Using: Read()
✓ Tool completed
✓ Tool completed
🤖 Using: Bash()
🤖 Using: Bash()
✓ Tool completed
✓ Tool completed
🤖 Using: Read()
🤖 Using: Read()
🤖 Using: Read()
✓ Tool completed
✓ Tool completed
✓ Tool completed
🤖 Using: Bash()
✓ Tool completed
🤖 Using: Bash()
✓ Tool completed
✓ Tool completed
🤖 Thinking...


In [ ]:
visualize_conversation(messages)

在这里，当我们的主智能体决定使用子智能体时，它将：
  1. 使用参数调用任务工具，例如：
  ```json
    {
      "description": "分析招聘影响",
      "prompt": "分析招聘5名工程师的财务影响...",
      "subagent_type": "financial-analyst"
    }
  ```
  2. 任务工具在独立上下文中执行子智能体
  3. 将结果返回给主首席助理智能体以继续处理

## 将所有功能整合在一起

现在让我们将我们看到的所有内容整合在一起。我们将要求我们的智能体确定招聘3名高级工程师的财务影响，并将他们的洞察写入`output_reports/hiring_decision.md`。这展示了上述所有功能：
- **Bash工具**：用于执行`hiring_impact.py`脚本来确定招聘新工程师的影响
- **记忆**：读取目录中的`CLAUDE.md`作为上下文，了解当前的预算、跑道、收入和其他相关信息
- **输出样式**：在`chief_of_staff_agent/.claude/output-styles`中定义的不同输出样式
- **自定义斜杠命令**：使用扩展为在`chief_of_staff_agent/.claude/commands`中定义的完整提示的快捷方式`/budget-impact`
- **子智能体**：我们的`/budget_impact`命令指导主助理智能体调用在`chief_of_staff_agent/.claude/agents`中定义的financial-analyst子智能体
- **钩子**：钩子在`chief_of_staff_agent/.claude/hooks`中定义并在`chief_of_staff_agent/.claude/settings.local.json`中配置
    - 如果我们的智能体之一正在更新财务报告，钩子应将此编辑/写入活动记录在`chief_of_staff_agent/audit/report_history.json`日志文件中
    - 如果财务分析师子智能体将调用`hiring_impact.py`脚本，这将记录在`chief_of_staff_agent/audit/tool_usage_log.json`日志文件中

- **计划模式**：如果您希望主助理为您制定计划供您在采取任何行动之前批准，请取消注释下面注释的行

为了让它准备好，我们已将智能体循环封装在一个python文件中，类似于我们在上一个笔记本中所做的。查看`chief_of_staff_agent`子目录中的agent.py文件。

总而言之，我们的`send_query()`函数接受4个参数（prompt、continue_conversation、permission_mode和output_style），其他所有内容都在智能体文件中设置，即：系统提示、最大轮数、允许的工具和工作目录。

为了更好地可视化这一切如何结合，请查看[Claude为我们制作的流程和架构图 :)](./chief_of_staff_agent/flow_diagram.md)

In [16]:
from chief_of_staff_agent.agent import send_query

result, messages = await send_query(
    "/budget-impact hiring 3 senior engineers. Save your insights by updating the 'hiring_decision.md' file in /output_reports or creating a new file there",
    # permission_mode="plan", # Enable this to use planning mode
    output_style="executive",
)

🤖 Thinking...
🤖 Using: Task()
🤖 Using: Bash()
✓ Tool completed
🤖 Using: Read()
🤖 Using: Read()
🤖 Using: Read()
✓ Tool completed
✓ Tool completed
✓ Tool completed
✓ Tool completed
🤖 Thinking...
🤖 Using: Write()
✓ Tool completed
🤖 Thinking...


In [ ]:
visualize_conversation(messages)

## 结论

我们演示了Claude Code SDK如何使您能够构建具有企业级功能的复杂多智能体系统。从使用Bash工具的基本脚本执行开始，我们逐步介绍了高级功能，包括通过CLAUDE.md实现的持久记忆、针对不同受众的自定义输出样式、战略规划模式、面向用户便利性的斜杠命令、用于护栏的合规钩子，以及用于专门任务的子智能体协调。

通过结合这些功能，我们创建了一个能够处理复杂执行决策工作流的AI主助理。该系统将财务分析委托给专门的子智能体，通过钩子维护审计跟踪，为不同利益相关者调整通信风格，并提供由数据驱动分析支持的可操作洞察。

高级智能体模式和多智能体编排这一基础为您构建生产就绪的企业系统做准备。在下一个笔记本中，我们将探索如何通过模型上下文协议(MCP)服务器将我们的智能体连接到外部服务，显著扩展其超越内置工具的能力。

下一步：[02_The_observability_agent.ipynb](02_The_observability_agent.ipynb) - 学习如何通过MCP使用自定义集成和外部数据源扩展您的智能体。